# Aperture Photometry

***

# Kernel Information

To run this notebook, please select the "Roman Calibration" kernel at the top right of your window.

# Imports
We are using standard libraries from the Numpy and Astropy stack.

- *astropy*
- *numpy*
- *copy*
- *matplotlib*
- *photutils* is an Astropy-affiliated package for photometry
- *roman_datamodels* opens and validates WFI data files
- *asdf* opens WFI data files
- *os* for checking if files exist
- *s3fs* streams data from Simple Storage Service (S3) buckets on Amazon Web Services (AWS)

In [ ]:
from astropy.table import Table
import asdf
import copy
import matplotlib.pyplot as plt
import numpy as np
from photutils.aperture import CircularAperture, aperture_photometry
import roman_datamodels as rdm
import os
import s3fs

# Introduction
This notebook explains how to perform **forced aperture photometry** (also known as *forced photometry* or *aperture photometry*) on Roman WFI images. Aperture photometry is useful for measuring the integrated fluxes for a set of specified source positions and aperture sizes. This technique is often valuable for extracting fluxes of known sources when it is impractical to fit for their positions and light distributions, or when simplicity and speed are high priorities. For example, aperture photometry can be used for:

- **Faint sources.** If a source is too faint, then it can be difficult to fit for the source centroid, light profile, and flux.
- **Sources detected at other wavelengths.** If a source is detected in a given bandpass, and the source size is expected to be the same across wavelengths, then it can be useful to define one aperture and measure the source fluxes across multiple filters.
- **Time-series evolution of sources.** If a source brightness is decreasing over time, then we can use earlier observations obtained when the source was bright to specify an aperture and extract the flux in a time series.

Here, we cover a simple example using the `photutils` package to extract fluxes from a catalog of known sources (stars and galaxies).

***

# Tutorial Data

## Image Data

In this tutorial, we use a Level 2 (L2; calibrated rate image) WFI data file that is the result of RomanCal processing of a Level 1 (L1; uncalibrated ramp cube) simulated file created with Roman I-Sim. If you have already worked through the tutorials "Simulating WFI Imaging Data with Roman I-Sim" and "Calibrating WFI Exposures with RomanCal," then you may already have these files saved locally. If not, then these files are also stored in the science platform S3 bucket. For more information on how to access these data, see the Data Discovery and Access tutorial.

As a reminder, the file we are using is a L2 file meaning that the data were processed to flag and/or correct for detector-level effects (e.g., saturation, classic non-linearity, etc.), and that the ramp was fitted into a count rate image in units of Data Numbers (DN) per second.

In [ ]:
#Stream the files from the S3 bucket if they are not in local storage

if os.path.exists('r0003201001001001004_0001_wfi01_f106_cal.asdf'):
    f = rdm.open('r0003201001001001004_0001_wfi01_f106_cal.asdf')
    image = f.data.copy()
    wcs = copy.deepcopy(f.meta.wcs)
else:
    fs = s3fs.S3FileSystem()
    asdf_dir_uri = 's3://roman-sci-test-data-prod-summer-beta-test/AAS_WORKSHOP/'
    asdf_file_uri = asdf_dir_uri + 'r0003201001001001004_0001_wfi01_f106_cal.asdf'
    with fs.open(asdf_file_uri, 'rb') as f:
        af = asdf.open(f)
        dm = rdm.open(af)
        image = dm.data.value.copy()
        wcs = copy.deepcopy(dm.meta.wcs)

## Source Catalog

We also have access to a source catalog that was used to simulate the WFI image. It contains stars and galaxies, which are labeled as `PSF` and `SER` under the column `type`. Source fluxes are available in all WFI filters (`F062`, `F087`, `F106`, `F129`, `F146`, `F158`, `F184`, `F213`) and are sampled from a lognormal distribution. Note that fluxes are all given in *maggies*, which are defined as ${\rm maggie} \equiv 10^{-0.4 m_{AB}}$, for an AB apparent magnitude $m_{AB}$. 

For galaxies, morphological parameters like `n` (Sersic index), `half_light_radius`, `pa` (position angle), and `ba` (axis ratio) are also provided in the catalog. These are sampled according to fiducial (and likely unrealistic) distributions.

In [ ]:
#Stream the files from the S3 bucket if they are not in local storage

if os.path.exists('full_catalog.ecsv'):
    cat = Table.read('full_catalog.ecsv')
else:
    fs = s3fs.S3FileSystem()
    asdf_dir_uri = 's3://roman-sci-test-data-prod-summer-beta-test/AAS_WORKSHOP/'
    asdf_file_uri = asdf_dir_uri + 'full_catalog.ecsv'
    with fs.open(asdf_file_uri, 'rb') as f:
        cat = Table.read(f, format='ascii.ecsv')

We can also display the first five rows and all columns of the catalog:

In [ ]:
cat[:5]

We can convert (RA, Dec) to (x, y) positions on the WFI01 detector.

In [ ]:
x_cat, y_cat = wcs.world_to_array_index_values(cat["ra"], cat["dec"])

It might be helpful to quantify the number of sources there are before we try to visualize them! We first create a mask of sources that actually fall on the detector, and then divide the remaining sources into stars and galaxies.

In [ ]:
on_detector_mask = (x_cat >= 0) & (x_cat <= 4095) & (y_cat >= 0) & (y_cat <= 4095)
print(f"Number of sources on detector: {sum(on_detector_mask)}")

stars = cat[on_detector_mask & (cat["type"] == "PSF")]
galaxies = cat[on_detector_mask & (cat["type"] == "SER")]
print(f"Number of stars: {len(stars)}\nNumber of galaxies: {len(galaxies)}")

Note that there are two orders of magnitude more stars than galaxies. We can now plot the distribution of source fluxes using a histogram. 

In [ ]:
fig, ax = plt.subplots()
ax.hist(np.log10(stars["F106"].value), bins=50, range=[-10, -5], log=True, label="Stars")
ax.hist(np.log10(galaxies["F106"].value), bins=50, range=[-10, -5], log=True, label="Galaxies")

ax.set_xlabel("$\log_{10}$(F106 flux [maggies])")
ax.set_ylabel("Number of sources")
ax.legend()

ax.grid(alpha=0.3)

# Forced Aperture Photometry

Forced aperture photometry is the process of using predefined source positions to place apertures and measure the flux within them. We use the source catalog information provided above for this. If source positions are unavailable, you will need to perform source detection first to determine their locations (see [Additional Resources](#Additional-Resources)).

## Create Apertures

From the catalogs, we now know the positions of every selected star and galaxy. We can define set aperture radii in units of pixels; we choose radii of 3 pixels for stars and 5 pixels for galaxies.

In [ ]:
star_positions = [(y, x) for x, y in zip(*wcs.world_to_array_index_values(stars["ra"], stars["dec"]))]
star_apertures = CircularAperture(positions=star_positions, r=3)

galaxy_positions = [(y, x) for x, y in zip(*wcs.world_to_array_index_values(galaxies["ra"], galaxies["dec"]))]
galaxy_apertures = CircularAperture(positions=galaxy_positions, r=5)

## Source Visualization

In [ ]:
fig, ax = plt.subplots(figsize=(9, 9))

# show the simulated image
ax.imshow(image, origin='lower', vmin=0, vmax=12, cmap="gray_r", )

# plot circles over bright galaxies and stars
star_apertures.plot(color="C0")
galaxy_apertures.plot(color="C1")

# zoom in on 1/16th of the image
ax.set_xlim(1024, 1536)
ax.set_ylim(1024, 1536)

plt.axis("off")
plt.show()

## Aperture Photometry with Photutils

We can now perform aperture photometry on the selected sources. Note that the input catalog contained all the sources in the region observed with the WFI, but not all the sources in the catalog fall necessarily on a WFI detector. In this case, the aperture photometry will have a value of NaN.

In [ ]:
star_phot = aperture_photometry(image, star_apertures)
star_phot

In [ ]:
galaxy_phot = aperture_photometry(image, galaxy_apertures)
galaxy_phot

Let's evaluate our results by plotting the measured fluxes versus fluxes in the simulated catalog. The blue points show the stars and follow a much tighter relation with respect to the extended sources, which is expected given their compact sizes and smaller apertures. On the other hand, galaxies require larger apertures, but in some cases (especially for brighter galaxies), they still lose flux and/or are contaminated by nearby sources, a problem that is often more noticeable for fainter galaxies.

In [ ]:
fig, ax = plt.subplots(figsize=(5, 5))
ax.scatter(stars["F106"], star_phot["aperture_sum"], s=5, c="C0", label="Stars", alpha=0.5)
ax.scatter(galaxies["F106"], galaxy_phot["aperture_sum"], s=5, c="C1", label="Galaxies", alpha=0.5)

# Set log scale on both axes
ax.set_xscale("log")
ax.set_yscale("log")

# Label the axes
ax.set_xlabel("F106 true flux (maggies)")
ax.set_ylabel("F106 Aperture flux (DN/s)")
ax.legend(loc="lower right", fontsize=12)

# Overplot a reference grid
ax.grid(alpha=0.3)

In addition to what we expected, we observe a distribution of stars with very low measured fluxes of around 20 DN/s across a wide range of input catalog fluxes. These sources were skipped in the Roman I-Sim simulation. As a result, although they appear in the input catalog, there is no corresponding source at those positions in the simulated image, meaning we are measuring sky background levels instead.

# Aditional Resources
The [Photutils documentation](https://photutils.readthedocs.io/en/stable/) has additional tutorials for detecting and fitting sources in images:

- [Aperture photometry](https://photutils.readthedocs.io/en/stable/aperture.html)
- [Background estimation](https://photutils.readthedocs.io/en/stable/background.html)
- [Source detection](https://photutils.readthedocs.io/en/stable/detection.html)
- [PSF photometry](https://photutils.readthedocs.io/en/stable/psf.html)

## About This Notebook

**Author:** John F. Wu, Tyler Desjardins\
**Updated On:** 2024-12-15

***

[Top of Page](#top)
<img style="float: right;" src="https://raw.githubusercontent.com/spacetelescope/notebooks/master/assets/stsci_pri_combo_mark_horizonal_white_bkgd.png" alt="Space Telescope Logo" width="200px"/> 